pip install pandas numpy scikit-learn spacy transformers torch matplotlib seaborn jupyter
python -m spacy download es_core_news_sm  # Modelo de español para spaCy

## Preprocesamiento del texto

Convierte archivos `.xlsx` en texto limpio para entrenar a los modelos.

In [10]:
import pandas as pd # Biblioteca para manipulación de datos
import re  # Biblioteca para expresiones regulares
#from sklearn.model_selection import train_test_split # Esta función se utiliza para dividir el conjunto de datos en conjuntos de entrenamiento y prueba.

# Cargar el conjunto de datos de entrenamiento, prueba y dev 
train_df = pd.read_excel('data/train.xlsx')
test_df = pd.read_excel('data/test.xlsx')
dev_df = pd.read_excel('data/development.xlsx')

# Función de limpieza
def limpia_texo(texto):
    texto = str(texto).lower()  # Convertir a minúsculas
    texto = re.sub(r'http\S+|ww\S+|@\w+|#\w+', '', texto)  # Eliminar URLs, menciones y hashtags
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar caracteres especiales
    return texto

# Crear una nueva columna 'clean_text' en el DataFrame 'train_df' aplicando la función 'limpia_texo' a la columna 'Text'
train_df['texto_limpio'] = train_df['Text'].apply(limpia_texo)
test_df['texto_limpio'] = test_df['TEXT'].apply(limpia_texo)
dev_df['texto_limpio'] = dev_df['Text'].apply(limpia_texo)

# Se corrige la diferencia de valores en la columna 'Category' del DataFrame 'test_df'
test_map = {True: 1, False: 0}
test_df['CATEGORY'] = test_df['CATEGORY'].map(test_map)
train_map = {"True": 1, "Fake": 0}
train_df['Category'] = train_df['Category'].map(train_map)
dev_df['Category'] = dev_df['Category'].map(train_map)

## Implementación de modelo Naive Bayes + TF-IDF

Se implementa el modelo Naive Bayes utilizanddo TF--IDF para la clasificación del texto. Este será el modelo clásico de referencia para comparar con los modelos de BERT y RoBERTa.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer # Convierte texto a vectores
from sklearn.naive_bayes import MultinomialNB # Clasificador Naive Bayes
from sklearn.metrics import classification_report # Genera un informe de clasificación del modelo

# Se vectoriza el texto limpio
vectorizer = TfidfVectorizer(max_features=5000) # Se limita a 5000 características
x_train = vectorizer.fit_transform(train_df['texto_limpio'])
x_test = vectorizer.transform(test_df['texto_limpio'])

# Se entrena al modelo
model_nb = MultinomialNB()
model_nb.fit(x_train, train_df['Category'])

# Se evalúa el modelo
y_pred = model_nb.predict(x_test)
reporte_nb = classification_report(test_df['CATEGORY'], y_pred) 
print(reporte_nb)

              precision    recall  f1-score   support

           0       0.67      0.53      0.60       286
           1       0.61      0.74      0.67       286

    accuracy                           0.64       572
   macro avg       0.64      0.64      0.63       572
weighted avg       0.64      0.64      0.63       572



## Implementación avanzado BERT

Se implementa el modelo BERT para la clasificación del texto. Este modelo es más avanzado, fine-tuning de un transformer, y se espera que tenga un mejor rendimiento en comparación con el modelo Naive Bayes.

Entrenamiento del modelo BERT.

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch # Biblioteca para trabajar con modelos de lenguaje preentrenados
from sklearn.metrics import accuracy_score # Métrica de precisión

# Carga el tokenizador y el modelo BERT
# Use a valid Hugging Face model identifier
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Crea un mapeo de etiquetas a valores numéricos
label_map = {'FAKE': 0, 'REAL': 1} 

# Función para tokenizar el texto
def tokenizar_texto(texto):
    return tokenizer(texto['texto_limpio'].tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')

train_tokens = tokenizar_texto(train_df)
dev_tokens = tokenizar_texto(dev_df)

# Se crea un Dataset personalizado para BERT
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokens.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
    

# Se crea el NewsDataset para entrenamiento y prueba
train_dataset = NewsDataset(train_tokens, train_df['Category'].tolist())
dev_dataset = NewsDataset(dev_tokens, dev_df['Category'].tolist())

# Se configuran los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',  # Directorio de salida
    num_train_epochs=3,  # Número de épocas de entrenamiento
    per_device_train_batch_size=8,  # Tamaño del batch
    eval_strategy="epoch",  # Estrategia de evaluación
)

# Función para calcular la precisión
def compute_metrics(pred):
    labels = pred.label_ids # Se obtienen las etiquetas verdaderas
    preds = pred.predictions.argmax(-1) # argmax devuelve el índice del valor máximo a lo largo de un eje
    acc = accuracy_score(labels, preds) # Se calcula la precisión de acuerdo a las etiquetas verdaderas y las predicciones
    return {'accuracy': acc}

# Se crea el entrenador
trainer = Trainer(
    model = model, # Modelo a entrenar
    args = training_args,
    train_dataset = train_dataset, # Conjunto de datos de entrenamiento
    eval_dataset = dev_dataset, # Conjunto de datos de evaluación
    compute_metrics = compute_metrics, # Función para calcular la precisión
)

trainer.train() # Se entrena el modelo

# Se guarda el modelo
# model.save_pretrained('model/modelo_preentrenado')
# tokenizer.save_pretrained('model/tokenizador_preentrenado')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\PC\OneDrive\Escritorio\Universidad\Semestre 10\AyPIT\DetectorFakeNews_AyPIT_2025-2\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\PC\AppData\Local\Temp\ipykernel_25004\1989014552.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.tokens.items()}


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.692104,0.518644
2,No log,0.599404,0.691525
3,No log,0.708296,0.674576


c:\Users\PC\OneDrive\Escritorio\Universidad\Semestre 10\AyPIT\DetectorFakeNews_AyPIT_2025-2\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\PC\AppData\Local\Temp\ipykernel_25004\1989014552.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.tokens.items()}
c:\Users\PC\OneDrive\Escritorio\Universidad\Semestre 10\AyPIT\DetectorFakeNews_AyPIT_2025-2\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\PC\AppData\Local\Temp\ipykernel_25004\1989014552.py:27: UserW

TrainOutput(global_step=255, training_loss=0.6090862199371936, metrics={'train_runtime': 634.1143, 'train_samples_per_second': 3.198, 'train_steps_per_second': 0.402, 'total_flos': 133397305067520.0, 'train_loss': 0.6090862199371936, 'epoch': 3.0})

Se ejecuta el entrenamiento del modelo BERT utilizando el conjunto de datos preprocesado. Se utiliza la biblioteca `transformers` de Hugging Face para cargar el modelo y realizar el fine-tuning. *******************AJUSTAR**********************

In [13]:
# Se evalúa el modelo
test_tokens = tokenizar_texto(test_df)
test_dataset = NewsDataset(test_tokens, test_df['CATEGORY'].tolist()) # Se crea el conjunto de datos de prueba
resultados = trainer.evaluate(test_dataset) # Se evalúa el modelo

print(resultados) # Se imprimen los resultados

c:\Users\PC\OneDrive\Escritorio\Universidad\Semestre 10\AyPIT\DetectorFakeNews_AyPIT_2025-2\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\PC\AppData\Local\Temp\ipykernel_25004\1989014552.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.tokens.items()}


{'eval_loss': 1.0626661777496338, 'eval_accuracy': 0.534965034965035, 'eval_runtime': 31.906, 'eval_samples_per_second': 17.928, 'eval_steps_per_second': 2.257, 'epoch': 3.0}


## Comparación de resultados
Se comparan los resultados de los modelos Naive Bayes y BERT utilizando métricas como la precisión, la recuperación y la puntuación F1.

In [ ]:
import pandas as pd 

data = {
    "Modelo": ["Naive Bayes", "BERT"],
    "Precision": [reporte_nb[0], resultados['eval_accuracy']],
}

resultados_df = pd.DataFrame(data) # Se crea un DataFrame con los resultados

display(resultados_df) # Se muestran los resultados

,Modelo,Precision
0,Naive Bayes,
1,BERT,0.534965
